In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import math
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
# TreeNode to build the tree
class TreeNode:
    def __init__(self, entropy, level, prediction, splitFeature = "", isLeaf = False):
        self.level = level
        self.entropy = entropy
        self.prediction = prediction
        self.isLeaf = isLeaf
        self.splitFeature = splitFeature
        
    def predict(self):
        return self.prediction

In [3]:
def cal_entropy(pi_count):
    '''
        returns calculated entropy
    '''
    ans = 0
    pi_sum = sum(pi_count)
    for pi in pi_count:
        ans += (pi/pi_sum)*math.log2(pi/pi_sum)
    if(ans==0):
        return ans
    return -ans

In [4]:
def find_gain_ratio(X,Y,f,f_values,info_org):
    
    '''
        returns the splits made if split with feature(f) 
        and corresponding gain ratio of the feature(f)
    '''
    
    d_sum = len(Y)
    info_f,split_info_f = 0,0
    splits_X,splits_Y = {},{}
    for fval in f_values:
        X1 = X[X[f]==fval]   # finding the X corresponding to fval of feature f
        Y1 = Y[X1.index]     # finding the Y corresponding to fval of feature f
        splits_X[fval] = X1  # Adding X and Y splits so that they can be recursively called
        splits_Y[fval] = Y1

        pi_count = list(Y1.value_counts())   # count of Y values after splitting with feature f for value fval
        info_f += (len(Y1)/d_sum)*cal_entropy(pi_count)   # Information required for the same
        split_info_f -= (len(Y1)/d_sum)*math.log2(len(Y1)/d_sum)  # Split info for the same                                      

    info_gain_f = info_org - info_f         # Info_gain_feature = Orignal_Info - Info. after split at f
    gain_ratio_f = info_gain_f/split_info_f
    
    return splits_X,splits_Y,gain_ratio_f

In [5]:
def decision_tree_implementation(X,Y,features,level):
    '''
        return the root of the decision tree built
    '''
    # Finding different values in each node
    pi_count = list(Y.value_counts())
    pi_index = list(Y.value_counts().index)
    info_org = cal_entropy(pi_count)
    
    node_result = Y.value_counts().idxmax()
    
    print("Level is: ",level)
    for i in range(len(pi_count)):
        print("Count of",pi_index[i],"is:",pi_count[i])
    print("Current Entropy is:",info_org)
    
    # Base condition for decision trees
    if(len(pi_count)==1 or len(features)==0):
        root = TreeNode(info_org, level, node_result, isLeaf = True)  # creating leaf node
        print("Reached Leaf Node\n")
        return root
    
    max_gain_ratio = 0
    split_feature = ""
    split_final_X, split_final_Y = {}, {}
    for f in features:
        f_values = X[f].unique()
        splits_X,splits_Y,gain_ratio_f = find_gain_ratio(X,Y,f,f_values,info_org)
        
        if(gain_ratio_f > max_gain_ratio):
            max_gain_ratio = gain_ratio_f
            split_feature = f
            split_final_X = splits_X
            split_final_Y = splits_Y
            
    print("Splitting on feature",split_feature,"with gain ratio",max_gain_ratio,"\n")
    
    new_feature = features.copy()
    new_feature.remove(split_feature)    # removing the feature
    
    # creating the tree Node
    root = TreeNode(info_org, level, node_result, split_feature)
    root.children = {}
    
    for split in split_final_X:
        # Recursively calling on all the splits and adding those splits to the children
        root.children[split] = decision_tree_implementation(split_final_X[split],split_final_Y[split],new_feature,level+1)
    return root

In [6]:
def helper(root, x):
    '''
        return the prediction for a single row
    '''
    if(root.isLeaf):             # If it is a pure node return the prediction
        return root.predict()
    
    f = root.splitFeature
    if(x[f] not in root.children):  # If the feature is not present in that node return the prediction(majority) of that node
        return root.predict()
    return helper(root.children[x[f]],x)   # Recursively call on the child node for the feature

def predict(root,X):
    '''
        return the prediction array for a given X
    '''
    prediction = []
    for i in range(len(X)):
        p = helper(root,X.iloc[i,:])   # helper to find prediction of ith row
        prediction.append(p)
    return np.array(prediction)

In [7]:
def function1(x,v1,v2,v3):
    '''
        returns labels
    '''
    if x<=v1:
        return 'a'
    elif x>v1 and x<=v2:
        return 'b'
    elif x>v2 and x<=v3:
        return 'c'
    else:
        return 'd'
    
def preprocess(X):
    '''
        returns labelled data based on values <=25%, <=50%, <=75% and >75%
    '''
    v1,v2,v3 = X.quantile(0.25),X.quantile(0.5),X.quantile(0.75)
    return X.apply(function1,args = [v1,v2,v3])

In [8]:
iris = datasets.load_iris()
X = pd.DataFrame(iris.data,columns = iris.feature_names)  # converting into Pandas dataFrame

# Adding the labelled data to make it discrete and removing the continuous data
for i in X.columns:
    X[i+'_labelled'] = preprocess(X[i])
    X.drop(i,inplace = True, axis = 1)
    
Y = pd.DataFrame(iris.target).iloc[:,0]      # converting the output in Pandas dataSeries
labels = iris.target_names

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state = 1)
# calling decision_tree_implementation on x_train and y_train
print("Tree in Depth-First Manner")
root = decision_tree_implementation(x_train,y_train,list(X.columns),0)

Tree in Depth-First Manner
Level is:  0
Count of 2 is: 41
Count of 0 is: 37
Count of 1 is: 34
Current Entropy is: 1.58071971384221
Splitting on feature petal length (cm)_labelled with gain ratio 0.6008507954091064 

Level is:  1
Count of 1 is: 15
Count of 2 is: 13
Current Entropy is: 0.996316519558962
Splitting on feature petal width (cm)_labelled with gain ratio 0.266575203380578 

Level is:  2
Count of 1 is: 12
Count of 2 is: 8
Current Entropy is: 0.9709505944546686
Splitting on feature sepal length (cm)_labelled with gain ratio 0.2836846600308684 

Level is:  3
Count of 1 is: 6
Current Entropy is: 0.0
Reached Leaf Node

Level is:  3
Count of 2 is: 7
Count of 1 is: 6
Current Entropy is: 0.9957274520849256
Splitting on feature sepal width (cm)_labelled with gain ratio 0.13702530817459677 

Level is:  4
Count of 2 is: 4
Count of 1 is: 3
Current Entropy is: 0.9852281360342516
Reached Leaf Node

Level is:  4
Count of 1 is: 1
Current Entropy is: 0.0
Reached Leaf Node

Level is:  4
Count o

In [9]:
# Testing the performance of the tree built on training and testing data

y_train_pred = predict(root, x_train)   # getting training predictions
y_test_pred = predict(root,x_test)   # getting testing predictions
print("Predicted values of on x_test")
print(y_test_pred)
train_confusion_matrix = confusion_matrix(y_train,y_train_pred)
test_confusion_matrix = confusion_matrix(y_test,y_test_pred)
print("Confusion Matrix on Training Data")
print(train_confusion_matrix)
print("Confusion Matrix on Testing Data")
print(test_confusion_matrix)

Predicted values of on x_test
[0 1 1 0 2 1 2 1 0 2 1 1 2 1 2 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 2 2 1 2 2 0 1
 0]
Confusion Matrix on Training Data
[[37  0  0]
 [ 0 30  4]
 [ 0  0 41]]
Confusion Matrix on Testing Data
[[11  2  0]
 [ 0 14  2]
 [ 0  0  9]]


In [10]:
# Implementation of the same decision tree built on OR function

import pandas as pd
import numpy as np
mat = {
    "X1" : [True,False,True,False],
    "X2" : [True,True,False,False],
    "Y" : [True,True,True,False]
}
df = pd.DataFrame(mat)
features = list(df.iloc[:,:-1].columns)
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
root = decision_tree_implementation(X,Y,features,0)

y_pred = predict(root,X)
print(confusion_matrix(Y,y_pred))

Level is:  0
Count of True is: 3
Count of False is: 1
Current Entropy is: 0.8112781244591328
Splitting on feature X1 with gain ratio 0.31127812445913283 

Level is:  1
Count of True is: 2
Current Entropy is: 0.0
Reached Leaf Node

Level is:  1
Count of True is: 1
Count of False is: 1
Current Entropy is: 1.0
Splitting on feature X2 with gain ratio 1.0 

Level is:  2
Count of True is: 1
Current Entropy is: 0.0
Reached Leaf Node

Level is:  2
Count of False is: 1
Current Entropy is: 0.0
Reached Leaf Node

[[1 0]
 [0 3]]
